# 自定义层
研究人员有发明专门用于处理图像、文本、序列数据和执行动态规划的层。未来，你会遇到或要自己发明一个现在深度学习框架中还不存在的层。这种情况下，你必须构建自定义层。

### 5.4.1 不带参数的层

首先我们定义一个没有任何参数的自定义层。下面的`CenteredLayer`类要从其输入中减去均值。要构建它，我们只需继承基础层类并实现前向传播功能。

In [3]:
import torch
import torch.nn.functional as F
from torch import nn

class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self, X):
        return X - X.mean()

验证它是否可以按预期工作

In [5]:
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

现在我们可以将层作为组件合并到更复杂的模型中

In [6]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

作为额外的健全性检查，我们可以在向该网络发送随机数据后，检查均值是否为0.由于我们处理的是浮点数，因为存储精度的原因，我们仍可能会看到一个非常小的非零数。

In [43]:
Y = net(torch.rand(4, 8))
Y.mean()

tensor(5.5879e-09, grad_fn=<MeanBackward0>)

# 5.4.2 带参数的层
下面我们继续定义有参数的层，这些参数可以通过训练进行调整。我们可以使用内置函数来创建参数，这些函数提供了一些基本的管理功能。比如管理访问、初始化、共享、保存和加载模型参数。这样做的好处之一是：我们不需要为每个自定义层编写自定义的序列化程序。   

现在，让我们实现自定义版本的全连接层。该层需要两个参数，一个用于表示权重，一个用于表示偏置项。在此实现中，我们使用修正线性单元作为激活函数。该层需要输入参数`in_units`和`units`，分别表示输入数和输出数。


In [44]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

接下来，我们实例化`MyLinear`类并访问其模型参数

In [45]:
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[-0.0363,  2.3050,  2.3008],
        [ 1.4355,  0.5750,  1.4591],
        [ 1.3738, -0.6216, -1.3568],
        [ 1.2252,  2.3752,  1.4367],
        [ 0.1729, -0.3042,  0.7738]], requires_grad=True)

我们可以使用自定义层直接执行前向传播计算

In [46]:
linear(torch.rand(2, 5))

tensor([[2.0939, 1.1921, 0.0000],
        [1.7980, 1.5581, 0.6179]])

我们可以使用自定义的层构建模型，就像使用内置的全连接层一样使用自定义层。

In [47]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[1.1367],
        [0.2759]])